In [10]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime as dt

In [75]:
weather_path = "../data/HourlyWeatherData.csv"
weatherDF = pd.read_csv(weather_path)
weatherDF


,DATE,HourlyDryBulbTemperature,HourlyPrecipitation,HourlyRelativeHumidity
0,2017-07-01T00:54:00,75.0,0.0,88.0
1,2017-07-01T01:54:00,76.0,0.0,88.0
2,2017-07-01T02:54:00,75.0,0.0,90.0
3,2017-07-01T03:54:00,75.0,0.0,90.0
4,2017-07-01T04:54:00,75.0,0.0,90.0
...,...,...,...,...
36307,2021-08-22T22:54:00,73.0,0.0,87.0
36308,2021-08-22T23:54:00,71.0,0.0,96.0
36309,2021-08-23T00:54:00,72.0,0.0,94.0
36310,2021-08-23T01:54:00,71.0,NaN,96.0


In [76]:
weatherDF['DATE'] = pd.to_datetime(weatherDF['DATE'])
weatherDF.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36312 entries, 0 to 36311
Data columns (total 4 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   DATE                      36312 non-null  datetime64[ns]
 1   HourlyDryBulbTemperature  36310 non-null  float64       
 2   HourlyPrecipitation       36109 non-null  float64       
 3   HourlyRelativeHumidity    36310 non-null  float64       
dtypes: datetime64[ns](1), float64(3)
memory usage: 1.1 MB


In [77]:
# Subtract minutes to get hour values on weather table
weatherDF['DATE'] = weatherDF['DATE'] - dt.timedelta(minutes = 54)
weatherDF

,DATE,HourlyDryBulbTemperature,HourlyPrecipitation,HourlyRelativeHumidity
0,2017-07-01 00:00:00,75.0,0.0,88.0
1,2017-07-01 01:00:00,76.0,0.0,88.0
2,2017-07-01 02:00:00,75.0,0.0,90.0
3,2017-07-01 03:00:00,75.0,0.0,90.0
4,2017-07-01 04:00:00,75.0,0.0,90.0
...,...,...,...,...
36307,2021-08-22 22:00:00,73.0,0.0,87.0
36308,2021-08-22 23:00:00,71.0,0.0,96.0
36309,2021-08-23 00:00:00,72.0,0.0,94.0
36310,2021-08-23 01:00:00,71.0,NaN,96.0


In [78]:
salespath = "../data/HourlySales1.csv" 
salesDF = pd.read_csv(salespath)
salesDF

,Business Date,Business Hour,Sales,Checks
0,7/1/2017,11:00:00,19.72,3
1,7/1/2017,11:00:00,9.68,1
2,7/1/2017,11:00:00,5.58,2
3,7/1/2017,12:00:00,19.12,3
4,7/1/2017,12:00:00,34.90,3
...,...,...,...,...
7935,12/21/2017,14:00:00,12.96,1
7936,12/21/2017,14:00:00,3.29,1
7937,12/21/2017,14:00:00,23.33,4
7938,12/21/2017,15:00:00,9.97,1


In [79]:
salesDF['DateTime'] = pd.to_datetime(salesDF['Business Date'] + " " + salesDF['Business Hour'])
salesDF = salesDF.drop(['Business Date','Business Hour'], axis=1)
salesDF

,Sales,Checks,DateTime
0,19.72,3,2017-07-01 11:00:00
1,9.68,1,2017-07-01 11:00:00
2,5.58,2,2017-07-01 11:00:00
3,19.12,3,2017-07-01 12:00:00
4,34.90,3,2017-07-01 12:00:00
...,...,...,...
7935,12.96,1,2017-12-21 14:00:00
7936,3.29,1,2017-12-21 14:00:00
7937,23.33,4,2017-12-21 14:00:00
7938,9.97,1,2017-12-21 15:00:00


In [80]:
groupedSalesDF = salesDF.groupby(['DateTime']).sum()
groupedSalesDF.reset_index(level=0, inplace=True)
groupedSalesDF

,DateTime,Sales,Checks
0,2017-07-01 11:00:00,34.98,6
1,2017-07-01 12:00:00,69.36,10
2,2017-07-01 13:00:00,72.88,6
3,2017-07-01 14:00:00,84.89,12
4,2017-07-01 15:00:00,103.16,5
...,...,...,...
2101,2017-12-21 12:00:00,48.39,6
2102,2017-12-21 13:00:00,32.19,3
2103,2017-12-21 14:00:00,39.58,6
2104,2017-12-21 15:00:00,9.97,1


In [86]:
WeatherSalesDF = groupedSalesDF.merge(weatherDF, how="inner", left_on="DateTime", right_on= "DATE")
WeatherSalesDF = WeatherSalesDF.drop(['DATE'], axis=1)
WeatherSalesDF

,DateTime,Sales,Checks,HourlyDryBulbTemperature,HourlyPrecipitation,HourlyRelativeHumidity
0,2017-07-01 11:00:00,34.98,6,86.0,0.0,65.0
1,2017-07-01 12:00:00,69.36,10,86.0,0.0,65.0
2,2017-07-01 13:00:00,72.88,6,90.0,0.0,59.0
3,2017-07-01 14:00:00,84.89,12,90.0,0.0,58.0
4,2017-07-01 15:00:00,103.16,5,89.0,0.0,57.0
...,...,...,...,...,...,...
2099,2017-12-21 12:00:00,48.39,6,48.0,0.0,37.0
2100,2017-12-21 13:00:00,32.19,3,49.0,0.0,36.0
2101,2017-12-21 14:00:00,39.58,6,49.0,0.0,36.0
2102,2017-12-21 15:00:00,9.97,1,48.0,0.0,37.0


In [87]:
WeatherSalesDF.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2104 entries, 0 to 2103
Data columns (total 6 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   DateTime                  2104 non-null   datetime64[ns]
 1   Sales                     2104 non-null   float64       
 2   Checks                    2104 non-null   int64         
 3   HourlyDryBulbTemperature  2104 non-null   float64       
 4   HourlyPrecipitation       2104 non-null   float64       
 5   HourlyRelativeHumidity    2104 non-null   float64       
dtypes: datetime64[ns](1), float64(4), int64(1)
memory usage: 115.1 KB
